In [76]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [77]:
df = pd.read_csv("Churn_Modelling.csv")

In [78]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [79]:
### Doing some feature engineering ###
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [80]:
df.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [81]:
### changing the categorical columns into numerical values ###
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])

In [82]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [86]:
### Encoding categorical variables
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()
onehot_encoder_geo = onehot_encoder.fit_transform(df[['Geography']]).toarray()
onehot_encoder_geo

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [87]:
onehot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [88]:
df1 = pd.DataFrame(onehot_encoder_geo, columns=onehot_encoder.get_feature_names_out(['Geography']))

In [89]:
df1

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [90]:
df = pd.concat([df.drop(["Geography"],axis=1),df1],axis=1)

In [91]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [93]:
# saving the onehot encoder in a pickle file
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder, file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder, file)

In [56]:
### Splitting the data into training and testing sets ###
X = df.drop('Exited', axis=1)
y = df['Exited']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
## scale this features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [59]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

## ANN Implementaion

In [60]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [61]:
X_train.shape

(8000, 12)

In [62]:
X_train.shape[1]

12

In [63]:
#A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
## Initialising the ANN

model = Sequential(
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)), # input layer connected to first hidden layer for input shape we need to specify number of features
        Dense(32,activation='relu'), # it is second hidden layer connected to first hidden layer we no need to specify input shape here
        Dense(1,activation='sigmoid') # output layer it is connected to second hidden layer
    ]
)

In [64]:
model.summary() # summary of the model so here we can see number of parameters in each layer and total parameters is 416 + 528 + 17 = 961

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [65]:
# we will interate the model now and we will use binary_crossentropy as we are doing binary classification and adam optimizer
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.001) # Adam optimizer is an optimization algorithm that can be used instead of the classical stochastic gradient descent procedure to update network weights iteratively based on training data.
loss=tensorflow.keras.losses.BinaryCrossentropy() # Binary crossentropy loss function is used for binary classification tasks, where the output is either 0 or 1.
loss

In [66]:
opt

In [67]:
# we will compile the model now and we will use binary_crossentropy as we are doing binary classification 
# compile the model means setting up the model with loss function, optimizer, and metrics to monitor during training Can we tell the model how to learn before training starts?”
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [68]:
## setting up TensorBoard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # log/fit is the directory where the log files will be stored 
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
# callback for tensorboard ensuring that logs are saved during training


In [69]:
# calling early stopping to stop the training when there is no improvement in val_loss for 5 consecutive epochs

early_stopper = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True) 
# Early stopping is a form of regularization used to avoid overfitting when training a machine learning model.
# patience parameter specifies the number of epochs with no improvement after which training will be stopped.
# restore_best_weights=True means that after stopping, the model weights will be restored to the best observed during training. 


In [70]:
## Training the model ###
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorboard_callback, early_stopper]
)


Epoch 1/100


250/250 [==============================] - 4s 8ms/step - loss: 0.4541 - accuracy: 0.8089 - val_loss: 0.3805 - val_accuracy: 0.8445
Epoch 2/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3727 - accuracy: 0.8512 - val_loss: 0.3536 - val_accuracy: 0.8525
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3509 - accuracy: 0.8568 - val_loss: 0.3494 - val_accuracy: 0.8590
Epoch 4/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3439 - accuracy: 0.8587 - val_loss: 0.3476 - val_accuracy: 0.8540
Epoch 5/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3387 - accuracy: 0.8610 - val_loss: 0.3428 - val_accuracy: 0.8600
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3350 - accuracy: 0.8643 - val_loss: 0.3422 - val_accuracy: 0.8595
Epoch 7/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3319 - accuracy: 0.8649 - val_loss: 0.3409 - val_accuracy: 0.8635
Epoch 8/100

In [71]:
print(type(y_train))
print(y_train.shape)
print(y_train.dtype)

print(type(y_test))
print(y_test.shape)
print(y_test.dtype)


<class 'pandas.core.series.Series'>
(8000,)
int64
<class 'pandas.core.series.Series'>
(2000,)
int64


In [72]:
model.save('model.h5') # saving the model in h5 format means the model architecture, weights, and training configuration (loss, optimizer) are all saved in a single file with .h5 extension.

d:\udemy generative AI\9- ANN project Implementation\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [73]:
## load the tensorboard extension means we can visualize the training process
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [74]:
%tensorboard --logdir logs/fit # meaning display the tensorboard logs from the specified log directory

Launching TensorBoard...